# Fetching ERA5 data to the fire counts csv file

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import xarray as xr
#import glob
#import netCDF4 as nc
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
#file_list=glob.glob('/Users/Peter 1/Desktop/Uni/HackathonUOS/data/MPI_near_surf_air_temp/*.nc')
#file_list=glob.glob('/Users/Peter 1/Desktop/Uni/HackathonUOS/data/north_america/*.h5')
#file_list=glob.glob('/Users/Peter 1/Desktop/Uni/HackathonUOS/data/era_total_precipitation_resolution500/*.nc')
#ds=xr.open_dataset(file_list[0])

In [104]:
fire_counts=xr.Dataset(pd.read_csv('/Users/Peter 1/Desktop/Uni/HackathonUOS/data/fire_counts_from_mcd64a1_meta.csv') )
precip = xr.open_mfdataset(f'/Users/Peter 1/Desktop/Uni/HackathonUOS/data/era_total_precipitation_resolution500/era5_total_precipitation_daily_*.nc', combine='by_coords').tp
t2m = xr.open_mfdataset(f'/Users/Peter 1/Desktop/Uni/HackathonUOS/data/era_temperature500/era5_2m_temperature_daily_*.nc', combine='by_coords').t2m


fire_counts['month']=fire_counts['ProductStartDay']//30+1


def getfloat(da, variable_key):
    return float(da[variable_key].values)

# for the fire_metadata, extract h,v,month and year as floats
def get_era_slice(da):
    h=getfloat(da, 'h')
    v=getfloat(da, 'v')
    month=getfloat(da, 'month')
    year=getfloat(da, 'year')
    return h,v,month, year


get_era_slice(fire_counts.sel(dim_0=0))

(27.0, 5.0, 8.0, 2006.0)

In [20]:
# extract grid cells that match tile (h,v) / year / month

def extract_mean_std(var,h,v,yy,mm):
    """
    Extract mean and std over grid cells that match tile.
        
    Parameter:
    ----------
        var : xarray data set 
            containing input variable 
        h : int
            horizontal index
        v : int
            vertical index
        yy : string
            year
        mm : string
            month
    
    Returns:
    -------
        mean : float
            mean over all grid cells
        std : float
            std over all grid cells
        
     
    """
    
    def convert_lat_lon(h,v):    
        ymax,xmin,T, R=10007555,-20015109,1111950,6371007.181
        lat=(ymax-v*T)/R 
        lon=(xmin+h*T)/(np.cos(lat)*R) *180/np.pi
        lat = lat * 180/np.pi
        return lat,lon
      
    lat0,lon0 = convert_lat_lon(h,v)   
    lat1,lon1 = convert_lat_lon(h+1,v+1) 
    lat2,lon2 = convert_lat_lon(h+1,v)   
    lat3,lon3 = convert_lat_lon(h,v+1) 

    latmin=np.min([lat0,lat1,lat2,lat3])
    latmax=np.max([lat0,lat1,lat2,lat3])
    lonmin=np.min([lon0,lon1,lon2,lon3])
    lonmax=np.max([lon0,lon1,lon2,lon3])
    time_str = yy + '-' + mm
    
    # extract wanted grid cells
    var = var.sel(lat=slice(latmin,latmax), lon=slice(lonmin,lonmax), time=time_str)
    
    # return mean and std
    return var.mean().values, var.std().values

In [45]:
# quick fix for the requirement of passing strings of mm and yyyy format to extract_mean_std
def monthify(flo):
    flo=int(flo)
    flo=str(flo)
    if len(flo)==1:
        flo='0'+flo
    return flo
def yearify(flo):
    flo=int(flo)
    flo=str(flo)
    return flo

In [85]:
#this loops over all entries in the table to find precip mean and std values
precip_mean=np.array([])
precip_std=np.array([])
for i in fire_counts['dim_0'].values:
    h,v,mm, yy=get_era_slice(fire_counts.sel(dim_0=i))
    if yy==2020.0:
        precip_mean=np.append(precip_mean, np.nan)
        precip_std=np.append(precip_std, np.nan)
        continue
    mean, std= extract_mean_std(precip,h,v,yearify(yy),monthify(mm))
    precip_mean=np.append(precip_mean, mean)
    precip_std=np.append(precip_std, std)

In [107]:

fire_counts['precip_mean']=(['dim_0'], precip_mean)
fire_counts['precip_std']=(['dim_0'], precip_std)

In [105]:
temp_mean=np.array([])
temp_std=np.array([])
for i in fire_counts['dim_0'].values:
    h,v,mm, yy=get_era_slice(fire_counts.sel(dim_0=i))
    if yy==2020.0:
        temp_mean=np.append(temp_mean, np.nan)
        temp_std=np.append(temp_std, np.nan)
        continue
    mean, std= extract_mean_std(t2m,h,v,yearify(yy),monthify(mm))
    temp_mean=np.append(temp_mean, mean)
    temp_std=np.append(temp_std, std)

In [109]:
fire_counts['temp_mean']=(['dim_0'], temp_mean)
fire_counts['temp_std']=(['dim_0'], temp_std)

array([298.97463989, 298.41253662, 275.57327271, ..., 284.91238403,
       280.64129639, 280.02087402])

In [110]:
fire_counts

<xarray.Dataset>
Dimensions:          (dim_0: 62422)
Coordinates:
  * dim_0            (dim_0) int64 0 1 2 3 4 5 ... 62417 62418 62419 62420 62421
Data variables:
    fname            (dim_0) object 'MCD64A1.A2006213.h27v05.006.201701719042...
    v                (dim_0) int64 5 5 5 5 5 5 5 5 5 5 5 ... 5 5 5 5 5 5 5 5 5 5
    h                (dim_0) int64 27 27 27 27 27 27 27 ... 21 21 21 21 21 21 21
    year             (dim_0) int64 2006 2006 2006 2006 ... 2019 2019 2019 2019
    BurnedCells      (dim_0) int64 343 0 131 102615 52 ... 54 2188 8712 3801 58
    MissingCells     (dim_0) int64 22312 20614 208829 ... 8035 38791 507265
    LandCells        (dim_0) int64 4507441 4507441 4507441 ... 5450678 5450678
    ValidLandCells   (dim_0) int64 4485129 4486827 4298612 ... 5411887 4943413
    ProductStartDay  (dim_0) int64 213 182 32 152 91 335 ... 60 91 305 335 32
    ProductEndDay    (dim_0) int64 243 212 59 181 120 365 ... 90 120 334 365 59
    month            (dim_0) int64 8 7 2 6 4 12 1 10 9 ... 8 1 5 10 3 4 11 12 2
    precip_mean      (dim_0) float64 0.003184 0.006732 ... 0.002715 0.0021
    precip_std       (dim_0) float64 0.005701 0.01237 ... 0.005493 0.003938
    temp_mean        (dim_0) float64 299.0 298.4 275.6 ... 284.9 280.6 280.0
    temp_std         (dim_0) float64 3.162 2.815 5.095 ... 5.884 4.336 4.918

In [112]:
pan=fire_counts.to_dataframe()

In [113]:
pan.to_csv('/Users/Peter 1/Desktop/Uni/HackathonUOS/data/fire_precip_temp.csv')

NameError: name 'pd' is not defined